### Downloading of the required libraries

In [1]:
pip install --upgrade torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade transformers


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade torchvision

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install faiss-cpu

### Importing the required Libraries

In [5]:
import transformers
from transformers import DPRContextEncoderTokenizer,DPRContextEncoder,DPRQuestionEncoderTokenizer,DPRQuestionEncoder

C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import numpy as np
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer,Pipeline
import pandas as pd

### Loading The Data (The Company Policies)

In [7]:
with open('companyPolicies.txt','r') as f:
    text_input=f.read()

In [8]:
text_input

"1.\tCode of Conduct\n\nOur Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.\nIntegrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.\nRespect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.\nAccountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices. We report any potential violati

In [9]:
import re

def split_para(text):

    raw = [p.strip() for p in text.split("\n\n") if p.strip()]

    merged = []
    i = 0

    while i < len(raw):
        block = raw[i]

        if re.match(r"^\d+\.\s*", block):
            if i + 1 < len(raw):
                combined = block + "\n" + raw[i + 1]
                merged.append(combined)
                i += 2
            else:
                merged.append(block)
                i += 1
        else:
            merged.append(block)
            i += 1

    return merged


In [10]:
text_input=split_para(text_input)
paragraphs=text_input

In [11]:
text_input

["1.\tCode of Conduct\nOur Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.\nIntegrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.\nRespect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.\nAccountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices. We report any potential violatio

### Loading The DPR Context Encoder

In [12]:
context_tokenizer=DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
context_tokenizer

DPRContextEncoderTokenizer(name_or_path='facebook/dpr-ctx_encoder-single-nq-base', vocab_size=30522, model_max_length=1000000000000000019884624838656, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [13]:
text=[("How are you","I am Fine"),("Code of Conduct","Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization.")]
tokenized_text=context_tokenizer(text,return_tensors='pt',padding=True,turnucation=True,max_length=256)
tokenized_text

C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\transformers\tokenization_utils_base.py:2299: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


{'input_ids': tensor([[  101,   100,  2024,  2017,   102,   100,  2572,   100,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0],
        [  101,   100,  1997,   100,   102,   100,   100,  1997,   100, 22106,
          1996,  8050,  6481,  1998, 12962,  4781,  2008,  5009,  2296,  2266,
          1997,  2256,  3029,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}

In [14]:
context_encoder=DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

Loading weights: 100%|█| 197/197 [00:00<00:00, 407.90it/s, Materializing param=ctx_encoder.bert_model.encoder.layer.11.
DPRContextEncoder LOAD REPORT from: facebook/dpr-ctx_encoder-single-nq-base
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
ctx_encoder.bert_model.pooler.dense.bias   | UNEXPECTED |  | 
ctx_encoder.bert_model.pooler.dense.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [15]:
encoded_text=context_encoder(**tokenized_text)

In [16]:
encoded_text

DPRContextEncoderOutput(pooler_output=tensor([[-0.1174,  0.6313, -0.5366,  ..., -0.4621,  0.5005,  0.0636],
        [ 0.4567,  0.8397, -0.3309,  ..., -0.4183,  0.3213,  0.1116]],
       grad_fn=<SelectBackward0>), hidden_states=None, attentions=None)

### Function to perform this encoding on Given Company Policies

In [17]:
def encode_context(text_input,context_tokenizer,context_encoder):
    tokenized_text=context_tokenizer(text_input,return_tensors='pt',padding=True,turnucation=True,max_length=256)
    encoded_text=context_encoder(**tokenized_text)
    return encoded_text

In [18]:
text_input=encode_context(text_input,context_tokenizer,context_encoder)

In [19]:
text_input

DPRContextEncoderOutput(pooler_output=tensor([[ 0.6779,  0.5649, -0.1093,  ..., -0.4969, -0.0482,  0.0861],
        [ 0.2972,  0.3041, -0.1318,  ..., -0.7091, -0.0412, -0.1221],
        [ 0.1854,  0.6510,  0.2969,  ..., -0.6377, -0.2537, -0.1587],
        ...,
        [ 0.7005,  0.4074, -0.0193,  ..., -0.4701, -0.1229, -0.0765],
        [ 0.1581,  0.5658, -0.0512,  ..., -0.5111, -0.5566, -0.0112],
        [ 0.1527,  0.3965,  0.2880,  ..., -0.3793, -0.3773, -0.0061]],
       grad_fn=<SelectBackward0>), hidden_states=None, attentions=None)

In [20]:
emb=text_input.pooler_output
emb.shape

torch.Size([9, 768])

### Making FAISS Instance

In [21]:
import faiss
embedding_dim=text_input.pooler_output.shape[1]   
embedding_dim

768

In [22]:
index=faiss.IndexFlatIP(embeddin_dim)

In [23]:
context_embeddings_np=np.array(text_input.pooler_output.detach()).astype('float32')
faiss.normalize_L2(context_embeddings_np)
index.add(context_embeddings_np)


C:\Users\rajve\AppData\Local\Temp\ipykernel_30068\1214308166.py:1: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  context_embeddings_np=np.array(text_input.pooler_output.detach()).astype('float32')


### Example Query and response using Faiss

In [45]:
querry="Code of Conduct"


In [46]:
question_tokenizer=DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
question_encoder=question_encoder=DPRQuestionEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

Loading weights: 0it [00:00, ?it/s]
DPRQuestionEncoder LOAD REPORT from: facebook/dpr-ctx_encoder-single-nq-base
Key                                                                                  | Status     | 
-------------------------------------------------------------------------------------+------------+-
ctx_encoder.bert_model.encoder.layer.{0...11}.output.LayerNorm.bias                  | UNEXPECTED | 
ctx_encoder.bert_model.encoder.layer.{0...11}.attention.output.LayerNorm.weight      | UNEXPECTED | 
ctx_encoder.bert_model.encoder.layer.{0...11}.intermediate.dense.weight              | UNEXPECTED | 
ctx_encoder.bert_model.encoder.layer.{0...11}.attention.output.LayerNorm.bias        | UNEXPECTED | 
ctx_encoder.bert_model.encoder.layer.{0...11}.attention.self.value.weight            | UNEXPECTED | 
ctx_encoder.bert_model.encoder.layer.{0...11}.output.dense.weight                    | UNEXPECTED | 
ctx_encoder.bert_model.encoder.layer.{0...11}.output.dense.bias                

In [47]:
tokenized_querry=question_tokenizer(querry,padding=True,turnucation=True,return_tensors='pt',max_len=256)
encoded_querry=question_encoder(**tokenized_querry).pooler_output.detach().numpy().astype('float32')

In [48]:

D, I = index.search(encoded_querry,k=5)


In [52]:
D,I=index.search(encoded_querry,k=3)

In [53]:
I

array([[8, 5, 7]])

In [55]:
for i in I[0]:
    print(paragraphs[i])

9.	Discipline and Termination Policy
The Discipline and Termination Policy underscores the organization's commitment to maintaining a productive, ethical, and respectful work environment. This policy applies to all personnel, including employees, contractors, and temporary staff.
Performance and Conduct Expectations: Employees are expected to meet performance standards and adhere to conduct guidelines. The organization will provide clear expectations, feedback, and opportunities for improvement when performance or conduct issues arise.
Disciplinary Actions: When necessary, disciplinary actions will be taken, which may include verbal warnings, written warnings, suspension, or other appropriate measures. Disciplinary actions are designed to address issues constructively and maintain performance standards.
Termination: In situations where an employee's performance or conduct issues persist, the organization may resort to termination. Termination may also occur for reasons such as redundan

In [56]:
D

array([[ 0.04292297, -0.16825289, -0.32728755]], dtype=float32)

### Loading GPT Generator

In [57]:
from transformers import pipeline

generator = pipeline(
    task="text-generation",
    model="openai-community/gpt2",
    max_new_tokens=100
)


Loading weights: 100%|██████████████████| 148/148 [00:00<00:00, 627.10it/s, Materializing param=transformer.wte.weight]
GPT2LMHeadModel LOAD REPORT from: openai-community/gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


### Testing It without giving context

In [58]:
context="What is Drug Policy"

In [59]:
generator(context)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': "What is Drug Policy? I don't know what that means, but it seems like a good place to start.\n\nThis list is by no means exhaustive. If you don't know what exactly you're looking for, please read this blog post.\n\nIn particular, I'm looking for a list of all of the major drugs by country that are illegal in the United States. I'll use any drug that is legal in the United States, and then I'll look at all of the drugs that are not"}]

### Giving context to Model

In [60]:
input1="What is Drug Policy"

In [61]:
input_tokenized=question_tokenizer(input1,return_tensors='pt',padding=True,turnucation=True,max_len=256)
input_encoded=question_encoder(**input_tokenized).pooler_output.detach().numpy().astype('float32')

In [62]:
D,I=index.search(input_encoded,k=1)

In [63]:
relevant = []

context_text = "\n".join(paragraphs[i] for i in I[0])

prompt = f"""
Use the context to answer the question in ONE sentence.
Do not Generate A follow up question The answer should be summary from context only , I don't want any irrelevant text

Context:
{context_text}

Question:
{input1}

Answer:
"""


In [64]:
prompt

"\nUse the context to answer the question in ONE sentence.\nDo not Generate A follow up question The answer should be summary from context only , I don't want any irrelevant text\n\nContext:\n9.\tDiscipline and Termination Policy\nThe Discipline and Termination Policy underscores the organization's commitment to maintaining a productive, ethical, and respectful work environment. This policy applies to all personnel, including employees, contractors, and temporary staff.\nPerformance and Conduct Expectations: Employees are expected to meet performance standards and adhere to conduct guidelines. The organization will provide clear expectations, feedback, and opportunities for improvement when performance or conduct issues arise.\nDisciplinary Actions: When necessary, disciplinary actions will be taken, which may include verbal warnings, written warnings, suspension, or other appropriate measures. Disciplinary actions are designed to address issues constructively and maintain performance 

In [65]:
generator(prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': "\nUse the context to answer the question in ONE sentence.\nDo not Generate A follow up question The answer should be summary from context only , I don't want any irrelevant text\n\nContext:\n9.\tDiscipline and Termination Policy\nThe Discipline and Termination Policy underscores the organization's commitment to maintaining a productive, ethical, and respectful work environment. This policy applies to all personnel, including employees, contractors, and temporary staff.\nPerformance and Conduct Expectations: Employees are expected to meet performance standards and adhere to conduct guidelines. The organization will provide clear expectations, feedback, and opportunities for improvement when performance or conduct issues arise.\nDisciplinary Actions: When necessary, disciplinary actions will be taken, which may include verbal warnings, written warnings, suspension, or other appropriate measures. Disciplinary actions are designed to address issues constructively and m

In [68]:
def rag_pipeline(question,generator, k=1):
    inputs = question_tokenizer(question, return_tensors="pt",
                                truncation=True, padding=True)

    outputs = question_encoder(**inputs)

    query_vec = (
        outputs.pooler_output
        .detach()
        .cpu()
        .numpy()
        .astype("float32")
    )

  
    D, I = index.search(query_vec, k)

    context = "\n".join(paragraphs[i] for i in I[0])

    prompt = f"""
    Use the context to answer the question in one or two sentences.

    Context:
    {context}

    Question:
    {question}

    Answer:
    """

    result = generator(prompt)

    return result[0]["generated_text"]


In [70]:
rag_pipeline(" Recruitment Process",generator=generator)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


"\n    Use the context to answer the question in one or two sentences.\n\n    Context:\n    9.\tDiscipline and Termination Policy\nThe Discipline and Termination Policy underscores the organization's commitment to maintaining a productive, ethical, and respectful work environment. This policy applies to all personnel, including employees, contractors, and temporary staff.\nPerformance and Conduct Expectations: Employees are expected to meet performance standards and adhere to conduct guidelines. The organization will provide clear expectations, feedback, and opportunities for improvement when performance or conduct issues arise.\nDisciplinary Actions: When necessary, disciplinary actions will be taken, which may include verbal warnings, written warnings, suspension, or other appropriate measures. Disciplinary actions are designed to address issues constructively and maintain performance standards.\nTermination: In situations where an employee's performance or conduct issues persist, th